In [1]:
!pip install -qU langchain openai langchain-community numexpr


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Getting Started with Chains

Chains are the core of LangChain. They are simply a chain of components, executed in a particular order.

The simplest of these chains is the `LLMChain`. It works by taking a user's input, passing in to the first element in the chain — a `PromptTemplate` — to format the input into a particular prompt. The formatted prompt is then passed to the next (and final) element in the chain — a LLM.

We'll start by importing all the libraries that we'll be using in this example.

In [2]:
import inspect
import re

from getpass import getpass
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback


To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted. 

In [3]:
# OPENAI_API_KEY = getpass() # SRA_DEBUGGING: Uncomment once testing is done.

In [4]:


# initialize the models
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-4" or other available models
    temperature=0.7
)

An extra utility we will use is this function that will tell us how many tokens we are using in each call. This is a good practice that is increasingly important as we use more complex tools that might make several calls to the API (like agents). It is very important to have a close control of how many tokens we are spending to avoid unsuspected expenditures.

In [5]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

## What are chains anyway?

Chains in LangChain are now built using the LangChain Expression Language (LCEL), which takes a declarative approach to combining components. Instead of using predefined chain classes, LCEL lets you compose chains using the `|` operator and other composition primitives.

### Types of Chain Composition

1. **Sequential Chains** (`|` operator)
   - Chain components one after another
   - Example: `prompt | llm | output_parser`

2. **Parallel Chains** (`RunnableParallel`)
   - Run multiple operations concurrently
   - Example: Running multiple prompts or retrievers in parallel

3. **Complex Workflows**
   - For more complex scenarios involving branching, cycles, or multiple agents
   - Recommended to use LangGraph instead of LCEL directly

Let's start with a simple example: creating a sequential math chain that can handle calculations...

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import numexpr

# Create a function to handle calculations
def calculate(expression: str) -> str:
    """Calculate using numexpr, with support for basic math operations."""
    try:
        result = float(numexpr.evaluate(expression))
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Create the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

# Create the chain using LCEL
math_chain = (
    prompt 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser()  # Convert to string
    | calculate  # Our calculation function
)

# Use the chain with our example
response = math_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)

The result is: 2.4116004626599237


Let's see what is going on here. The chain processes our input through several sequential steps:

1. The prompt template formats our question
2. The LLM converts it to a mathematical expression
3. The StrOutputParser ensures we get a clean string
4. Finally, our calculate function computes the result

But how did the LLM know to return just the mathematical expression? 🤔

**Enter prompts**

The question we send isn't the only input the LLM receives 😉. Look at our prompt template:


In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

The system message explicitly instructs the LLM to return only the mathematical expression. Without this context, the LLM would try to calculate the result itself. Let's test this by trying without the system message:


In [8]:
# Simple prompt without guidance
prompt = ChatPromptTemplate.from_messages([
    ("user", "{question}")
])

basic_chain = (
    prompt 
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

response = basic_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)  # The LLM tries to calculate it directly and gets it wrong!


13 raised to the 0.3432 power is approximately 3.144.


This demonstrates the power of prompting in LCEL: by carefully designing our prompts, we can guide the LLM's behavior precisely.

The beauty of LCEL's sequential composition is how clearly we can see each step in the chain:


In [9]:
math_chain = (
    prompt                         # Step 1: Format the input with our system message
    | ChatOpenAI(temperature=0)    # Step 2: Get mathematical expression from LLM
    | StrOutputParser()            # Step 3: Convert to clean string
    | calculate                    # Step 4: Evaluate the expression
)

Each step flows naturally into the next using the `|` operator, making it easy to understand and modify the chain's behavior. This is much more flexible than the old approach of using predefined chain classes - we can easily add, remove, or modify steps as needed!

*_Note: The `calculate` function uses `numexpr` to safely evaluate mathematical expressions without needing a full Python REPL (Read-Eval-Print Loop)._

### Building Complex Chains with LCEL

Let's build a more complex example that shows how to combine different components using LCEL. We'll create a chain that cleans up messy text and then paraphrases it in a specific style.

First, let's create a function to clean up text by removing extra spaces and newlines. In LCEL, we can use regular functions directly in our chain:

In [10]:
def clean_text(text: str) -> str:
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text

Now, let's create our prompt template for the paraphrasing:

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writing assistant."),
    ("user", """Please paraphrase this text in the style of {style}:

{text}""")
])

Now we can combine everything into a sequential chain using LCEL's `|` operator. The beauty of LCEL is how naturally we can compose these components:


In [12]:
# Create the chain using LCEL
style_chain = (
    {
        "text": lambda x: clean_text(x["text"]),  # Extract and clean the text from input dict
        "style": lambda x: x["style"]  # Extract style from input dict
    }
    | prompt  # Format with our template
    | ChatOpenAI(temperature=0.7)  # Generate creative paraphrase
    | StrOutputParser()  # Convert to string
)

# Our input text with messy spacing
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""

# Run the chain
response = style_chain.invoke({
    "text": input_text,
    "style": "a 90s rapper"
})
print(response)

Yo, check it – chains in the mix,
Bringin' together pieces quick,
Like a puzzle, makin' it slick,
One app, one vibe, one click.

User input start the flow,
PromptTemplate style it, let it show,
LLM takes it, makes it grow,
Chains connect, makin' moves, yo.

Complex chains, we ain't playin',
Combining links, they stay swayin',
Mix it up, keep it innovatin',
Chains and components, we stay creatin'.


Let's look at how this chain works:

1. The dictionary `{"text": clean_text, "style": lambda x: x}` processes our inputs in parallel using `RunnableParallel`
2. The `|` operator connects each component, showing the clear flow of data
3. Each step in the chain serves a specific purpose and is easily modifiable
4. The components work together seamlessly to process and transform the text

This demonstrates how LCEL lets us compose simple components into powerful chains while keeping the code readable and maintainable. Whether you're processing text, generating content, or building complex workflows, LCEL's composition primitives make it easy to build exactly what you need! 🔥

That's it for this example on chains.

---